In [1]:
from transformers import AutoTokenizer
from simple_loader import FilePathDataset, build_dataloader
from datasets import load_from_disk
import yaml, pickle

/home/PL-BERT/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
multi_lingual_tokenizer = AutoTokenizer.from_pretrained("bert-base-multilingual-cased")

/home/PL-BERT/venv/lib/python3.10/site-packages/huggingface_hub/file_download.py:797: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [10]:
multi_lingual_tokenizer.decode([72617])

'toren'

In [3]:
# unique_ids = []

# for token in token_maps:
#     index = token_maps[token]['token']

#     unique_ids.append(index)

# unique_ids = list(set(unique_ids))

# # Sort the list
# unique_ids.sort()

# is_consecutive = unique_ids == list(range(min(unique_ids), max(unique_ids) + 1))

# # Print results
# print(f"unique_ids are {'consecutive' if is_consecutive else 'not consecutive'}")
# print("Missing numbers (if any):", set(range(min(unique_ids), max(unique_ids) + 1)) - set(unique_ids))


In [4]:
# max(unique_ids)

NameError: name 'unique_ids' is not defined

In [5]:
# from itertools import chain

# keys = set(token_maps.keys())  # Convert to set for faster lookup
# for ds in hindi_dataset:
#     flattened_list = list(chain.from_iterable(ds['input_ids']))
#     missing_elements = set(flattened_list) - keys
    
#     if missing_elements:
#         print("Elements in flattened_list but not in token_maps keys:", missing_elements)
#         print("Count of missing elements:", len(missing_elements))

In [7]:
# from itertools import chain

# keys = set(token_maps.keys())  # Convert to set for faster lookup

# for ds in hindi_dataset:
#     flattened_list = list(chain.from_iterable(ds['input_ids']))
#     missing_elements = set(flattened_list) - keys
    
#     if missing_elements:
#         print("Count of missing elements:", len(missing_elements))
#         print("Elements in flattened_list but not in token_maps keys:", missing_elements)


In [9]:
hindi_dataset = load_from_disk("/home/PL-BERT/data/multilingual-pl-bert/hi")

In [4]:
dd = FilePathDataset(hindi_dataset)

In [5]:
loader = build_dataloader(hindi_dataset, num_workers=32, batch_size=128)

/home/ubuntu/PL-BERT/venv/lib/python3.10/site-packages/torch/utils/data/dataloader.py:617: UserWarning: This DataLoader will create 32 worker processes in total. Our suggested max number of worker in current system is 8, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


In [40]:

with open("/home/PL-BERT/new_token_maps.pkl", 'rb') as handle:
    token_maps = pickle.load(handle)

In [41]:
token_maps[28631]

{'word': '##מל', 'token': 94551}

In [7]:
config_path = "/home/ubuntu/PL-BERT/multilingual-pl-bert/config.yml" # you can change it to anything else
config = yaml.safe_load(open(config_path))

In [8]:
config

{'log_dir': 'Checkpoint_all_phonemes',
 'mixed_precision': 'fp16',
 'data_folder': 'wikipedia_20220301.en.processed',
 'batch_size': 32,
 'save_interval': 20000,
 'log_interval': 10,
 'num_process': 1,
 'num_steps': 2000000,
 'dataset_params': {'tokenizer': 'bert-base-multilingual-cased',
  'token_separator': ' ',
  'token_mask': 'M',
  'word_separator': 102,
  'token_maps': 'token_maps.pkl',
  'max_mel_length': 512,
  'word_mask_prob': 0.15,
  'phoneme_mask_prob': 0.1,
  'replace_prob': 0.2},
 'model_params': {'vocab_size': 178,
  'hidden_size': 768,
  'num_attention_heads': 12,
  'intermediate_size': 2048,
  'max_position_embeddings': 512,
  'num_hidden_layers': 12,
  'dropout': 0.1}}

In [9]:
special_token = config['dataset_params']['word_separator']

In [10]:
from itertools import chain

In [11]:
# get all unique tokens in the entire dataset

from tqdm import tqdm

unique_index = [special_token]
for _, batch in enumerate(tqdm(loader)):
    flattened_batch = list(chain.from_iterable(batch))
    flattened_batch = list(set(flattened_batch))
    # for ids in batch:
    unique_index.extend(flattened_batch)
    unique_index = list(set(unique_index))

100%|██████████| 269/269 [00:21<00:00, 12.47it/s]


In [29]:
unique_ids = []

for token in token_maps:
    index = token_maps[token]['token']

    unique_ids.append(index)

unique_ids = list(set(unique_ids))

# Sort the list
unique_ids.sort()

# Check if all elements are consecutive
are_consecutive = all(unique_ids[i] + 1 == unique_ids[i + 1] for i in range(len(unique_ids) - 1))

# Output result
print("All numbers are consecutive:", are_consecutive)
print("Max: ", max(unique_ids))

All numbers are consecutive: False
Max:  94550


In [13]:
# redo the mapping for lower number of tokens

for t in tqdm(unique_index):
    if t in token_maps.keys():
        continue

    word = multi_lingual_tokenizer.decode([t])
    token_maps[t] = {'word': word, 'token': unique_index.index(t) + max(unique_ids)}


100%|██████████| 34616/34616 [00:04<00:00, 7117.87it/s] 


In [14]:
len(token_maps)

79818

In [39]:
import pickle
with open("new_token_maps.pkl", 'wb') as handle:
    pickle.dump(token_maps, handle)
# print('Token mapper saved to %s' % config['dataset_params']['token_maps'])

In [16]:
# tokens_not_available = []

In [17]:
# for t in tqdm(unique_index):
#     try: 
#         token_maps[t]
#         pass
#     except:
#         print(multi_lingual_tokenizer.decode([t]))

In [18]:
from mydataloader import build_dataloader, FilePathDataset

In [25]:
myds = FilePathDataset(hindi_dataset, tokenizer=multi_lingual_tokenizer, token_maps="new_token_maps.pkl")

177


In [27]:
loader = build_dataloader(hindi_dataset,batch_size=1
,dataset_config={
    "tokenizer": multi_lingual_tokenizer, 
    "token_maps":"new_token_maps.pkl",
})

177


In [29]:
for batch in loader:
    words, labels, phonemes, input_lengths, masked_indices = batch
    break
    

In [31]:
phonemes

tensor([[ 16,  46, 156,  57, 158,  16,  50, 156, 138,  90,  43, 158, 125,  16,
          62, 156,  51, 158,  56,  16,  83,  47,  57, 158, 136,  16,  44, 156,
         138,  56,  51,  16,  50, 156, 102,  56,  46,  51,  16,  44, 162, 156,
          43, 158, 130,  43, 158,  16,  53,  51,  16,  58, 162, 156, 102,  54,
          55,  83,  16,  50,  86, 158,  16,  58, 156,  63, 158,  60,  56,  65,
         102, 125, 156,  43, 158,  55,  16,  61,  83, 112,  53, 131, 156,  47,
         158,  58,  16,  45,  83, 125, 156, 102,  62, 125,  83,  16,  55, 156,
         135,  53, 162,  52,  83,  16,  53, 157,  83,  54,  43, 158,  53, 156,
          43, 158, 125,  16, 157, 138,  44, 162, 102, 130, 156,  47, 158,  53,
          16,  44, 156, 138,  45, 158,  83,  56,  16,  54, 156,  43, 158, 125,
          43, 158,  16,  46, 156, 138,  62, 158,  43, 158,  16,  45, 156, 138,
         112,  53,  51,  16,  58, 156,  43,  37,  80,  47, 158,  16,  37,  16,
          61, 156, 138, 114,  90,  83, 157,  86,  52

In [37]:
words, phonemes

(tensor([[ 1424,  6697,  6697,  6697,  6697,  6697,  6697,  6697,  6697,  6697,
           6697,  6697,  6697,  6697,  6697,  6697,  6697,  6697,  6697,  1424,
          61857, 61857, 61857, 61857, 61857,  1424, 90475, 90475, 90475, 90475,
          90475,  1424, 66712, 66712, 66712, 66712, 66712, 66712,  1424, 62954,
          62954, 62954, 62954, 62954, 62954, 62954, 62954,  1424, 62085, 62085,
           1424, 76541, 76541, 76541, 76541, 76541, 76541, 76541,  1424, 61890,
          61890, 61890,  1424, 60380, 60380, 60380, 60380, 60380, 60380, 60380,
          60380, 60380, 60380, 60380, 60380, 60380,  1424, 71302, 71302, 71302,
          73013, 73013, 73013, 62641, 62641, 66551,  1424,   368,  7635,  7635,
          77870, 77870, 77870, 77870, 77870,  1424, 72408, 72408, 72408, 72408,
          72408, 72408, 72408,  1424, 93676, 93676, 93676, 93676, 93676, 93676,
          93676, 93676, 93676, 93676, 93676,  1424,   363,   363, 34433,  8522,
           8522, 23608, 23608, 80329, 80